# Precision-Recall Tradeoff

このノートブックでは、PrecisionとRecallのトレードオフ関係を詳しく学習し、実務での活用方法を理解します。

## 学習目標
- Precision-Recall tradeoffの理論的理解
- 閾値の調整による指標の変化
- 実務での最適な閾値の選択
- Precision-Recall曲線の解釈


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (precision_recall_curve, precision_score, recall_score,
                           f1_score, average_precision_score, classification_report)
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. Precision-Recall Tradeoffの基本概念


In [ ]:
def calculate_precision_recall_manual(y_true, y_pred_proba, thresholds):
    """
    Precision-Recall曲線の手動実装
    
    Parameters:
    y_true: 実際のラベル
    y_pred_proba: 予測確率
    thresholds: 閾値のリスト
    
    Returns:
    results: 各閾値でのPrecisionとRecall
    """
    results = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        # 混同行列の計算
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        
        # PrecisionとRecallの計算
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        results.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    return results

# 簡単な例でPrecision-Recall tradeoffを理解
np.random.seed(42)
y_true_example = np.random.binomial(1, 0.3, 100)
y_pred_proba_example = np.random.beta(2, 5, 100)

# 異なる閾値での評価
thresholds_example = np.arange(0.1, 1.0, 0.05)
results_example = calculate_precision_recall_manual(y_true_example, y_pred_proba_example, thresholds_example)

# 結果の可視化
df_example = pd.DataFrame(results_example)

plt.figure(figsize=(15, 5))

# Precision vs Threshold
plt.subplot(1, 3, 1)
plt.plot(df_example['threshold'], df_example['precision'], 'o-', color='blue', label='Precision')
plt.xlabel('Threshold')
plt.ylabel('Precision')
plt.title('Precision vs Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

# Recall vs Threshold
plt.subplot(1, 3, 2)
plt.plot(df_example['threshold'], df_example['recall'], 'o-', color='red', label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Recall')
plt.title('Recall vs Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

# Precision vs Recall
plt.subplot(1, 3, 3)
plt.plot(df_example['recall'], df_example['precision'], 'o-', color='green', label='PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision vs Recall')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

print("=== Precision-Recall Tradeoffの基本概念 ===")
print("Precision: 正例と予測したもののうち、実際に正例だった割合")
print("Recall: 実際の正例のうち、正しく正例と予測できた割合")
print("\nトレードオフの関係:")
print("- 閾値を下げる → Recall ↑, Precision ↓")
print("- 閾値を上げる → Precision ↑, Recall ↓")
print("- 理想: 両方とも高い値")
print("- 現実: 一方を上げると他方が下がる")


## 2. 実データでのPrecision-Recall曲線


In [ ]:
# 乳がんデータセットでのPrecision-Recall曲線
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ロジスティック回帰モデルの訓練
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 予測確率
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Precision-Recall曲線の計算
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
average_precision = average_precision_score(y_test, y_pred_proba)

# 可視化
plt.figure(figsize=(15, 5))

# Precision-Recall曲線
plt.subplot(1, 3, 1)
plt.plot(recall, precision, 'b-', linewidth=2, label=f'PR Curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True, alpha=0.3)

# Precision vs Threshold
plt.subplot(1, 3, 2)
plt.plot(thresholds, precision[:-1], 'b-', linewidth=2, label='Precision')
plt.xlabel('Threshold')
plt.ylabel('Precision')
plt.title('Precision vs Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

# Recall vs Threshold
plt.subplot(1, 3, 3)
plt.plot(thresholds, recall[:-1], 'r-', linewidth=2, label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Recall')
plt.title('Recall vs Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=== 乳がんデータセットでのPrecision-Recall曲線 ===")
print(f"Average Precision: {average_precision:.4f}")
print(f"データセットの情報:")
print(f"  サンプル数: {len(y_test)}")
print(f"  クラス分布: {np.bincount(y_test)}")
print(f"  正例の割合: {np.mean(y_test):.2%}")

# 異なる閾値での性能
def evaluate_threshold_performance(y_true, y_pred_proba, thresholds):
    """
    異なる閾値での性能評価
    """
    results = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        
        results.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    return results

# 異なる閾値での評価
thresholds_eval = np.arange(0.1, 1.0, 0.1)
results_eval = evaluate_threshold_performance(y_test, y_pred_proba, thresholds_eval)

# 結果の表示
df_eval = pd.DataFrame(results_eval)
print(f"\n=== 異なる閾値での性能 ===")
print(df_eval.round(4))

# 最適な閾値の選択
best_f1_idx = df_eval['f1'].idxmax()
best_threshold = df_eval.loc[best_f1_idx, 'threshold']
best_f1 = df_eval.loc[best_f1_idx, 'f1']

print(f"\n最適なF1-score: {best_f1:.4f} (閾値: {best_threshold:.1f})")


## 3. 実務での閾値選択


In [ ]:
# 実務での閾値選択戦略
def find_optimal_threshold(y_true, y_pred_proba, strategy='f1'):
    """
    実務での最適な閾値の選択
    
    Parameters:
    y_true: 実際のラベル
    y_pred_proba: 予測確率
    strategy: 選択戦略 ('f1', 'precision', 'recall', 'balanced')
    
    Returns:
    optimal_threshold: 最適な閾値
    """
    thresholds = np.arange(0.1, 1.0, 0.01)
    results = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        
        results.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    df = pd.DataFrame(results)
    
    if strategy == 'f1':
        optimal_idx = df['f1'].idxmax()
    elif strategy == 'precision':
        optimal_idx = df['precision'].idxmax()
    elif strategy == 'recall':
        optimal_idx = df['recall'].idxmax()
    elif strategy == 'balanced':
        # PrecisionとRecallのバランスを取る
        df['balance'] = np.abs(df['precision'] - df['recall'])
        optimal_idx = df['balance'].idxmin()
    
    return df.loc[optimal_idx, 'threshold'], df.loc[optimal_idx]

# 異なる戦略での最適な閾値
strategies = ['f1', 'precision', 'recall', 'balanced']
optimal_thresholds = {}

for strategy in strategies:
    threshold, metrics = find_optimal_threshold(y_test, y_pred_proba, strategy)
    optimal_thresholds[strategy] = {
        'threshold': threshold,
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'f1': metrics['f1']
    }

# 結果の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 各戦略の比較
strategies_data = []
for strategy, data in optimal_thresholds.items():
    strategies_data.append({
        'Strategy': strategy,
        'Threshold': data['threshold'],
        'Precision': data['precision'],
        'Recall': data['recall'],
        'F1': data['f1']
    })

df_strategies = pd.DataFrame(strategies_data)

# 閾値の比較
axes[0, 0].bar(df_strategies['Strategy'], df_strategies['Threshold'], color='skyblue')
axes[0, 0].set_title('Optimal Thresholds by Strategy')
axes[0, 0].set_ylabel('Threshold')
axes[0, 0].tick_params(axis='x', rotation=45)

# Precisionの比較
axes[0, 1].bar(df_strategies['Strategy'], df_strategies['Precision'], color='lightcoral')
axes[0, 1].set_title('Precision by Strategy')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].tick_params(axis='x', rotation=45)

# Recallの比較
axes[1, 0].bar(df_strategies['Strategy'], df_strategies['Recall'], color='lightgreen')
axes[1, 0].set_title('Recall by Strategy')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].tick_params(axis='x', rotation=45)

# F1-scoreの比較
axes[1, 1].bar(df_strategies['Strategy'], df_strategies['F1'], color='gold')
axes[1, 1].set_title('F1-score by Strategy')
axes[1, 1].set_ylabel('F1-score')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("=== 実務での閾値選択戦略 ===")
print(df_strategies.round(4))

# 実務での解釈
print("\n=== 実務での解釈 ===")
print("医療診断:")
print("  - Recall重視: 病気の見逃しを避ける")
print("  - 低い閾値: より多くの患者を検査対象に")

print("\nスパム判定:")
print("  - Precision重視: 重要なメールの誤判定を避ける")
print("  - 高い閾値: 確実なスパムのみを判定")

print("\n推薦システム:")
print("  - F1-score重視: バランスの取れた性能")
print("  - 中程度の閾値: 精度と網羅性のバランス")


## 4. 演習問題

### 演習1: 異なるデータセットでの分析
クラス不均衡データでPrecision-Recall曲線を描画し、閾値の影響を観察してみましょう。

### 演習2: カスタム戦略の実装
ビジネス要件に応じたカスタム閾値選択戦略を実装してみましょう。

### 演習3: 複数モデルの比較
異なるモデルでのPrecision-Recall曲線を比較し、性能を評価してみましょう。


## まとめ

このノートブックでは、Precision-Recall tradeoffについて詳しく学習しました。

**学習した内容**：
- Precision-Recall tradeoffの理論的理解
- 閾値の調整による指標の変化
- 実務での最適な閾値の選択戦略
- Precision-Recall曲線の解釈

**重要なポイント**：
- PrecisionとRecallはトレードオフの関係
- 実務では目的に応じた閾値選択が重要
- 複数の戦略を比較して最適解を見つける
- 可視化により直感的な理解が可能

**次のステップ**：
- ROC曲線とAUCの学習
- 多クラス分類の評価指標
- より高度な評価手法の学習
